In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split


import joblib

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

pd.options.display.max_rows = 4000

# Load Data

In [30]:
# import data
# tracker
# site

def read_all_csv_files(root_folder):
    df_tracker = pd.DataFrame()
    df_site = pd.DataFrame()
    popular_tracker = []
    for dirpath,dirnames,filenames in os.walk(root_folder):
        temp = dirpath.split('\\')
        country = temp[-1]
        for filename in filenames:
            if filename == 'sites.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df['country']=country

                df_site = pd.concat([df_site,df])
                
            if filename == 'sites_trackers.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df['country']=country

                df_tracker = pd.concat([df_tracker,df])
            
            if filename == 'trackers.csv':
                path = os.path.join(dirpath,filename)
                df = pd.read_csv(path)
                df_popular_tracker = df[df['reach']>0.04]['tracker']
                popular_tracker=popular_tracker+ (list(df_popular_tracker))
                popular_tracker = list(set(popular_tracker))

                

    return df_tracker,df_site,popular_tracker




In [31]:
df_tracker, df_site, popular_tracker = read_all_csv_files('C:/Users/ywang/Desktop/CSE6242/project/train')

In [ ]:
popular_tracker = []
with open('popular_tracker.txt', 'r') as file:
    for line in file:
        popular_tracker.append(line.strip())

# Preprocess Data

In [5]:
# get site_tracker

df_tracker = df_tracker[df_tracker.tracker.isin(popular_tracker)].copy()

df_tracker_site=pd.get_dummies(df_tracker,prefix = None,columns = ['tracker'])

In [6]:
df_tracker_site=df_tracker_site[['site','month']+['tracker_'+x for x in list(df_tracker.tracker.drop_duplicates())]].copy().drop_duplicates()

In [7]:
df_tracker_site = df_tracker_site.groupby(['site','month']).sum()

In [8]:
df_tracker_site.columns = [x.replace('tracker_','') for x in df_tracker_site.columns]

In [9]:
df_site['de'] = df_site.country.apply(lambda x: 1 if x == 'de' else 0)
df_site['eu'] = df_site.country.apply(lambda x: 1 if x == 'eu' else 0)
df_site['fr'] = df_site.country.apply(lambda x: 1 if x == 'fr' else 0)
df_site['global'] = df_site.country.apply(lambda x: 1 if x == 'global' else 0)
df_site['us'] = df_site.country.apply(lambda x: 1 if x == 'us' else 0)

In [10]:
df_site = df_site.drop(['Unnamed: 0','country'],axis=1)

In [11]:
df_site = pd.get_dummies(df_site,columns=['category']).set_index(['site','month'])

In [12]:
df_model = df_site.join(df_tracker_site,how='left')

df_model.head()

,,popularity,cookies,bad_qs,tracked,https,requests,requests_tracking,content_length,requests_failed,has_blocking,script,iframe,beacon,image,stylesheet,font,xhr,plugin,media,referer_leaked,referer_leaked_header,referer_leaked_url,cookie_samesite_none,t_active,hosts,trackers,companies,de,eu,fr,global,us,category_Adult,category_Banking,category_Business,category_E-Commerce,category_Entertainment,category_Government,category_Health,category_News and Portals,category_Political,category_Recreation,category_Reference,category_Uncategorized,google_tag,amazon_cloudfront,google_analytics,google,gstatic,facebook,google_fonts,youtube,googleapis.com,cloudflare,criteo,doubleclick,appnexus,outbrain,adobe_audience_manager,amazon_web_services,google_adservices,google_syndication,jsdelivr,google_users,scorecard_research_beacon,amazon_adsystem,onetrust,bing_ads,twitter,amazon_cdn,google_photos,reddit,didomi,cloudflare_insights,usercentrics,at_internet,qualtrics,cloudinary,forter
site,month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
google.com,2024-07,1.000000,0.236459,0.006271,0.237701,0.998492,59.616008,2.743598,1.907268e+06,7.577944,0.176041,0.918758,0.176933,0.037532,0.927501,0.600016,0.830353,0.318307,0.000021,0.064175,0.123900,0.078331,0.021010,0.104942,153441.602015,3.564908,3.236687,1.555520,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,True,False,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amazon.de,2024-07,0.748776,0.083797,0.000739,0.084015,0.998964,190.921843,0.370541,2.143226e+07,12.360872,0.547009,0.978160,0.892779,0.034383,0.839610,0.728406,0.626472,0.722753,0.000329,0.245649,0.185830,0.183222,0.002111,0.077044,125816.192463,4.381320,3.142395,1.469835,1,0,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
youtube.com,2024-07,0.659274,0.538790,0.002785,0.539198,0.999716,429.905406,3.181029,2.683666e+06,244.525943,0.460755,0.985727,0.143418,0.026340,0.982907,0.788637,0.864587,0.991561,0.000191,0.004635,0.369505,0.287898,0.041668,0.074934,798469.793256,7.989726,7.004539,1.224744,1,0,0,0,0,False,False,False,False,True,False,False,False,False,False,False,False,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kleinanzeigen.de,2024-07,0.348909,0.046207,0.022716,0.048292,0.999851,18.200140,3.675121,1.860285e+05,4.354929,0.300112,0.991996,0.112785,0.050082,0.292424,0.040956,0.030441,0.409647,0.000382,0.000738,0.162493,0.039395,0.115601,0.033365,72867.828997,5.339930,5.142943,4.257678,1,0,0,0,0,False,False,False,False,False,False,False,False,False,False,False,True,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ebay.de,2024-07,0.242525,0.620532,0.009258,0.621190,0.968622,66.006635,3.770803,2.422414e+06,6.871002,0.622415,0.996841,0.471117,0.052388,0.961508,0.661477,0.559307,0.348965,0.000247,0.003560,0.213119,0.182730,0.023080,0.065238,101619.041530,4.378217,2.148025,1.998829,1,0,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
feature = pd.read_csv('feature.csv')

# feature is a data frame contains the name of features and their importance. The importance is from previous model trained and we will use it to reduce the dimension of the features.
feature


,columns,importance
0,trackers,0.059464
1,companies,0.054357
2,hosts,0.044788
3,font,0.039795
4,iframe,0.039672
5,content_length,0.038398
6,beacon,0.038167
7,bad_qs,0.037881
8,requests_failed,0.037629
9,requests_tracking,0.037215


In [14]:
# We only choose features with importance >= 0.01
important_feature = list(feature[feature['importance']>=0.01]['columns'])


trackers = list(df_tracker.tracker.drop_duplicates())
X = df_model[important_feature].copy()
X_columns = X.columns
y = df_model[trackers]


X=X.fillna(0).astype('float').to_numpy()
y=y.fillna(0).astype('float').to_numpy()

In [39]:
important_feature

['trackers',
 'companies',
 'hosts',
 'font',
 'iframe',
 'content_length',
 'beacon',
 'bad_qs',
 'requests_failed',
 'requests_tracking',
 'cookie_samesite_none',
 'requests',
 'xhr',
 'referer_leaked_url',
 'stylesheet',
 'tracked',
 'image',
 'media',
 'referer_leaked',
 'cookies',
 'referer_leaked_header',
 'has_blocking',
 'script',
 't_active',
 'popularity',
 'https']

In [15]:
# delete dataframes to release memory

del df_site
del df_model
del df_tracker
del df_tracker_site

In [16]:
# Split dataset to training dataset and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model and check the performance

In [17]:
multi_label_rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [18]:
multi_label_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
y_pred=multi_label_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     42820
           1       0.94      0.67      0.78     13510
           2       0.88      0.97      0.92     35228
           3       0.97      0.99      0.98     45333
           4       0.98      0.99      0.99     46860
           5       0.89      0.91      0.90     24497
           6       0.93      0.99      0.96     43427
           7       0.88      0.90      0.89     23753
           8       0.90      0.98      0.94     39772
           9       0.88      0.80      0.84     21840
          10       0.94      0.80      0.86     13649
          11       0.94      0.97      0.95     36280
          12       0.94      0.81      0.87     14257
          13       0.98      0.53      0.69      5899
          14       0.93      0.74      0.82     10919
          15       0.91      0.63      0.75     16313
          16       0.87      0.90      0.88     21655
          17       0.92    

c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Check the importance again to see if we can further reduce the dimesion

In [20]:
importance = multi_label_rf.feature_importances_

feature_v2 = pd.DataFrame({'columns':X_columns,'importance':importance}).sort_values(by = 'importance',ascending= False).reset_index()

feature_v2

,index,columns,importance
0,6,beacon,0.065279
1,1,companies,0.060471
2,0,trackers,0.059216
3,9,requests_tracking,0.050416
4,3,font,0.048194
5,2,hosts,0.044793
6,4,iframe,0.042833
7,17,media,0.039766
8,10,cookie_samesite_none,0.039558
9,5,content_length,0.039366


# Tune the hyperparameter

In [21]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 20, 40],
    "min_samples_leaf": [1, 2, 4]
}
base_rf = RandomForestClassifier(random_state=42, n_jobs=-1)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(base_rf, param_grid, scoring="f1_micro", cv=cv, verbose=0)



In [22]:
grid.fit(X_train,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             param_grid={'max_depth': [None, 20, 40],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [100, 200]},
             scoring='f1_micro')

In [23]:
grid.best_params_

{'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}

In [25]:
model_t = grid.best_estimator_

In [27]:
# test the performance after tuned the hyperparameter

y_pred_t=model_t.predict(X_test)
print(classification_report(y_test, y_pred_t))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     42820
           1       0.94      0.67      0.79     13510
           2       0.88      0.97      0.92     35228
           3       0.97      0.99      0.98     45333
           4       0.98      0.99      0.99     46860
           5       0.89      0.91      0.90     24497
           6       0.93      0.99      0.96     43427
           7       0.88      0.90      0.89     23753
           8       0.90      0.98      0.94     39772
           9       0.89      0.81      0.84     21840
          10       0.94      0.80      0.86     13649
          11       0.94      0.97      0.96     36280
          12       0.94      0.81      0.87     14257
          13       0.98      0.53      0.69      5899
          14       0.93      0.74      0.82     10919
          15       0.91      0.63      0.75     16313
          16       0.87      0.90      0.88     21655
          17       0.92    

c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Test models using latest data

In [ ]:
df_tracker, df_site, _ = read_all_csv_files('C:/Users/ywang/Desktop/CSE6242/project/test')

popular_tracker = []
with open('popular_tracker.txt', 'r') as file:
    for line in file:
        popular_tracker.append(line.strip())

# get site_tracker

df_tracker = df_tracker[df_tracker.tracker.isin(popular_tracker)].copy()

df_tracker_site=pd.get_dummies(df_tracker,prefix = None,columns = ['tracker'])

df_tracker_site=df_tracker_site[['site','month']+['tracker_'+x for x in list(df_tracker.tracker.drop_duplicates())]].copy().drop_duplicates()

df_tracker_site = df_tracker_site.groupby(['site','month']).sum()

df_tracker_site.columns = [x.replace('tracker_','') for x in df_tracker_site.columns]

df_site['de'] = df_site.country.apply(lambda x: 1 if x == 'de' else 0)
df_site['eu'] = df_site.country.apply(lambda x: 1 if x == 'eu' else 0)
df_site['fr'] = df_site.country.apply(lambda x: 1 if x == 'fr' else 0)
df_site['global'] = df_site.country.apply(lambda x: 1 if x == 'global' else 0)
df_site['us'] = df_site.country.apply(lambda x: 1 if x == 'us' else 0)


df_site = df_site.drop(['Unnamed: 0','country'],axis=1)

df_site = pd.get_dummies(df_site,columns=['category']).set_index(['site','month'])

df_model = df_site.join(df_tracker_site,how='left')

# We only choose features with importance >= 0.01
important_feature = list(feature[feature['importance']>=0.01]['columns'])


trackers = list(df_tracker.tracker.drop_duplicates())
X = df_model[important_feature].copy()
X_columns = X.columns
y = df_model[trackers]


X=X.fillna(0).astype('float').to_numpy()
y=y.fillna(0).astype('float').to_numpy()

# delete dataframes to release memory

del df_site
del df_model
del df_tracker
del df_tracker_site

In [38]:
print(trackers)

['google_tag', 'amazon_cloudfront', 'cloudflare', 'googleapis.com', 'gstatic', 'google_analytics', 'outbrain', 'google_fonts', 'facebook', 'youtube', 'google', 'google_syndication', 'amazon_adsystem', 'doubleclick', 'amazon_web_services', 'appnexus', 'jsdelivr', 'criteo', 'google_users', 'adobe_audience_manager', 'bing_ads', 'onetrust', 'twitter', 'google_adservices', 'reddit', 'google_photos', 'amazon_cdn', 'usercentrics', 'cloudflare_insights', 'scorecard_research_beacon', 'didomi', 'at_internet', 'qualtrics', 'cloudinary', 'forter']


# Check the performance of first model

In [33]:
y_pred=multi_label_rf.predict(X)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.98      0.93     81164
           1       0.77      0.30      0.43     24186
           2       0.45      0.88      0.60     41893
           3       0.79      0.95      0.86     75547
           4       0.95      0.99      0.97     91074
           5       0.70      0.51      0.59     60775
           6       0.15      0.98      0.26     13990
           7       0.93      0.34      0.49     85678
           8       0.50      0.95      0.66     44786
           9       0.51      0.40      0.45     41090
          10       0.99      0.18      0.30     85682
          11       0.52      0.87      0.65     40315
          12       0.72      0.58      0.64     19997
          13       0.98      0.02      0.04     65942
          14       0.47      0.13      0.21     30718
          15       0.51      0.19      0.28     25837
          16       0.48      0.45      0.47     33468
          17       0.56    

c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Check the performance of the tuned model

In [36]:
y_pred_t=model_t.predict(X)
print(classification_report(y, y_pred_t))

              precision    recall  f1-score   support

           0       0.89      0.98      0.93     81164
           1       0.78      0.30      0.43     24186
           2       0.45      0.88      0.60     41893
           3       0.79      0.95      0.86     75547
           4       0.95      0.99      0.97     91074
           5       0.69      0.51      0.59     60775
           6       0.15      0.98      0.26     13990
           7       0.93      0.34      0.50     85678
           8       0.50      0.96      0.66     44786
           9       0.52      0.41      0.46     41090
          10       0.99      0.18      0.30     85682
          11       0.52      0.87      0.65     40315
          12       0.72      0.58      0.64     19997
          13       0.98      0.02      0.04     65942
          14       0.47      0.13      0.21     30718
          15       0.52      0.19      0.28     25837
          16       0.48      0.45      0.47     33468
          17       0.56    

c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ywang\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Save Model

In [37]:
joblib.dump(model_t,'trackers_model.joblib')

['trackers_model.joblib']